In [2]:
# -*- coding: utf-8 -*-


from __future__ import division, unicode_literals
from collections import Counter
import math, random

from ch04_linear_algebra import distance, vector_subtract, scalar_multiply

In [3]:
# 8.1. 경사 하강법에 숨은 의미

def sum_of_squares(v):
    """ Computes the sum of squared elements in v """
    return sum(v_i ** 2 for v_i in v)

In [5]:
# 8.3. 경사 적용하기

def step(v, direction, step_size):
    """ move step_size in the direction from v"""

    return [v_i + step_size * direction_i
            for v_i, direction_i in zip(v, direction)]


def sum_of_squares_gradient(v):
    return [2 * v_i for v_i in v]

In [6]:
# 8.4. 적절한 스텝 크기 정하기

def safe(f):
    """define a new function that wraps f and return it"""
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')         # this means "infinity" in Python
    return safe_f

In [7]:
# 8.5. 전부 조합하기

def minimize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):#특정모델의 파라미터에 대힌 오류값 함수,그에 대한 미분함수
    """use gradient descent to find theta that minimizes target function"""

    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

    theta = theta_0                           # set theta to initial value  #(초기값)
    target_fn = safe(target_fn)               # safe version of target_fn # 안전한(오류없는) 함수인가 판정
    value = target_fn(theta)                  # value we're minimizing 

    while True:
        gradient = gradient_fn(theta)
        next_thetas = [step(theta, gradient, -step_size)
                       for step_size in step_sizes]

        # choose the one that minimizes the error function
        next_theta = min(next_thetas, key=target_fn)
        next_value = target_fn(next_theta)

        # stop if we're "converging"
        if abs(value - next_value) < tolerance:
            return theta
        else:
            theta, value = next_theta, next_value

In [8]:
def negate(f):
    """return a function that for any input x returns -f(x)"""
    return lambda *args, **kwargs: -f(*args, **kwargs)


def negate_all(f):
    """the same when f returns a list of numbers"""
    return lambda *args, **kwargs: [-y for y in f(*args, **kwargs)]


def maximize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    return minimize_batch(negate(target_fn),
                          negate_all(gradient_fn),
                          theta_0,
                          tolerance)

In [9]:
# 8.6. SGD

def in_random_order(data):
    """generator that returns the elements of data in random order"""
    indexes = [i for i, _ in enumerate(data)]  # create a list of indexes
    random.shuffle(indexes)                    # shuffle them
    for i in indexes:                          # return the data in that order
        yield data[i]


def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):

    data = zip(x, y)
    theta = theta_0                             # initial guess
    alpha = alpha_0                             # initial step size
    min_theta, min_value = None, float("inf")   # the minimum so far
    iterations_with_no_improvement = 0

    # if we ever go 100 iterations with no improvement, stop
    while iterations_with_no_improvement < 100:
        value = sum( target_fn(x_i, y_i, theta) for x_i, y_i in data )

        if value < min_value:
            # if we've found a new minimum, remember it
            # and go back to the original step size
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
        else:
            # otherwise we're not improving, so try shrinking the step size
            iterations_with_no_improvement += 1
            alpha *= 0.9

        # and take a gradient step for each of the data points
        for x_i, y_i in in_random_order(data):
            gradient_i = gradient_fn(x_i, y_i, theta)
            theta = vector_subtract(theta, scalar_multiply(alpha, gradient_i))

    return min_theta

In [14]:
def maximize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):
    return minimize_stochastic(negate(target_fn),
                               negate_all(gradient_fn),
                               x, y, theta_0, alpha_0)


if __name__ == "__main__":
    print ("using the gradient")

    v = [random.randint(-10,10) for i in range(3)]
    tolerance = 0.0000001
    print(v) #(-10,-10,8)
    while True:
        #print v, sum_of_squares(v)
        gradient = sum_of_squares_gradient(v)   # compute the gradient at v (-20,-20,16)
        next_v = step(v, gradient, -0.01)       # take a negative gradient step [-9.8, -9.8, 7.84]
        if distance(next_v, v) < tolerance:     # stop if we're converging 
            break
        v = next_v                              # continue if we're not
        print(v)
    print(v)
    print ("minimum v", v)
    print ("minimum value", sum_of_squares(v))
    print ()

    print ("using minimize_batch")

    v = [random.randint(-10,10) for i in range(3)]

    v = minimize_batch(sum_of_squares, sum_of_squares_gradient, v)

    print ("minimum v", v)
    print ("minimum value", sum_of_squares(v))

using the gradient
[-10, -10, 8]
[-9.8, -9.8, 7.84]
[-9.604000000000001, -9.604000000000001, 7.6832]
[-9.41192, -9.41192, 7.529536]
[-9.2236816, -9.2236816, 7.37894528]
[-9.039207968000001, -9.039207968000001, 7.2313663744]
[-8.858423808640001, -8.858423808640001, 7.086739046912]
[-8.6812553324672, -8.6812553324672, 6.94500426597376]
[-8.507630225817856, -8.507630225817856, 6.806104180654285]
[-8.337477621301499, -8.337477621301499, 6.669982097041199]
[-8.17072806887547, -8.17072806887547, 6.536582455100375]
[-8.00731350749796, -8.00731350749796, 6.405850805998368]
[-7.847167237348001, -7.847167237348001, 6.2777337898784005]
[-7.690223892601041, -7.690223892601041, 6.152179114080832]
[-7.5364194147490196, -7.5364194147490196, 6.0291355317992155]
[-7.385691026454039, -7.385691026454039, 5.908552821163231]
[-7.237977205924959, -7.237977205924959, 5.7903817647399665]
[-7.0932176618064595, -7.0932176618064595, 5.674574129445167]
[-6.95135330857033, -6.95135330857033, 5.5610826468562635]
[-